In [45]:
import pandas as pd
train = pd.read_csv("train.csv",index_col=0)
test = pd.read_csv("test.csv",index_col=0)
sample_submit = pd.read_csv("sample_submit.csv",index_col=0,header=None)

In [46]:
train.head()

,age,sex,bmi,children,smoker,region,charges
id,,,,,,,
0,26,male,32.665465,3,no,southeast,0
1,41,male,29.798725,1,no,southwest,0
2,28,male,32.722029,0,yes,northwest,1
3,20,female,38.429831,2,no,southeast,0
4,45,female,29.641854,1,no,northwest,0


In [47]:
test.head()

,age,sex,bmi,children,smoker,region
id,,,,,,
13,19,male,24.365178,1,no,northeast
23,59,male,33.997763,0,yes,northeast
27,42,female,29.283450,0,no,southwest
28,30,male,24.903725,1,no,northwest
29,41,female,29.644536,0,no,southwest


In [48]:
print(train.shape)
print(test.shape)

(1600, 7)
(400, 6)


In [49]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1600 non-null   int64  
 1   sex       1600 non-null   object 
 2   bmi       1600 non-null   float64
 3   children  1600 non-null   int64  
 4   smoker    1600 non-null   object 
 5   region    1600 non-null   object 
 6   charges   1600 non-null   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 100.0+ KB


In [50]:
train['charges'].value_counts()

0    1256
1     198
2     146
Name: charges, dtype: int64

In [51]:
train.describe()

,age,bmi,children,charges
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,38.985000,32.424376,1.014375,0.306250
std,13.555012,5.766915,1.259031,0.628656
min,18.000000,20.627626,0.000000,0.000000
25%,27.000000,28.634267,0.000000,0.000000
50%,40.000000,32.268786,1.000000,0.000000
75%,50.000000,37.069581,2.000000,0.000000
max,64.000000,47.290644,5.000000,2.000000


In [52]:
train[['sex','charges']].groupby(['sex']).mean()

,charges
sex,
female,0.227931
male,0.376932


In [53]:
train[['smoker','charges']].groupby(['smoker']).mean()

,charges
smoker,
no,0.070579
yes,1.182891


In [54]:
train[['region','charges']].groupby(['region']).mean()

,charges
region,
northeast,0.277778
northwest,0.230769
southeast,0.465686
southwest,0.245333


In [55]:
data = pd.concat([train,test], sort=False)

In [56]:
data['sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['smoker'].replace(['no', 'yes'], [0, 1], inplace=True)
data['region'] = data['region'].map({'northeast': 0, 'northwest': 1, 'southeast': 2, 'southwest': 3}).astype(int)

In [57]:
data.head(10)

,age,sex,bmi,children,smoker,region,charges
id,,,,,,,
0,26,0,32.665465,3,0,2,0.0
1,41,0,29.798725,1,0,3,0.0
2,28,0,32.722029,0,1,1,1.0
3,20,1,38.429831,2,0,2,0.0
4,45,1,29.641854,1,0,1,0.0
5,20,0,37.785937,2,1,2,1.0
6,37,0,43.393374,2,0,0,0.0
7,41,1,32.280407,1,0,0,0.0
8,40,0,28.865828,1,0,0,0.0


In [58]:
data.shape

(2000, 7)

In [59]:
data.reset_index(drop=True, inplace=True)
train = data.loc[:(len(train)-1)]
test = data.loc[len(train):]

In [60]:
y_train = train['charges']
X_train = train.drop('charges', axis=1)
X_test = test.drop('charges', axis=1)

In [61]:
X_train.shape

(1600, 6)

In [62]:
X_test.shape

(400, 6)

In [63]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                                 random_state=0, stratify=y_train)

In [64]:
from sklearn.metrics import log_loss
import lightgbm as lgb
import numpy as np
import optuna
import tensorflow as tf

categorical_features = ['sex', 'smoker', 'region']

def objective(trial):
    params = {
        'objective': 'multiclass',
        'verbose': -1,
        'num_class': 3,
        'max_bin' : trial.suggest_int('max_bin',255,500),
        'learning_rate' : 0.05,
        'num_leaves' : trial.suggest_int('num_leaves',32,128),
        'metric' : 'multi_logloss',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
    }
    
    lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)
    
    model = lgb.train(params,lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      verbose_eval=10,
                      num_boost_round=1000,
                      early_stopping_rounds=10)
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid,y_pred_valid)
    return score

In [65]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective,n_trials=40)

[I 2023-06-19 21:52:26,239] A new study created in memory with name: no-name-5fc864f7-4068-4643-80cb-dd58f2dfcd10
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/rele

Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.490049	valid_1's multi_logloss: 0.50784
[20]	training's multi_logloss: 0.425426	valid_1's multi_logloss: 0.448819
[30]	training's multi_logloss: 0.379504	valid_1's multi_logloss: 0.412643
[40]	training's multi_logloss: 0.356326	valid_1's multi_logloss: 0.398345
[50]	training's multi_logloss: 0.339328	valid_1's multi_logloss: 0.390235
[60]	training's multi_logloss: 0.327231	valid_1's multi_logloss: 0.386616
[70]	training's multi_logloss: 0.317327	valid_1's multi_logloss: 0.384243
[80]	training's multi_logloss: 0.308475	valid_1's multi_logloss: 0.383741
Early stopping, best iteration is:
[72]	training's multi_logloss: 0.315772	valid_1's multi_logloss: 0.383481
Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.436274	valid_1's multi_logloss: 0.501209
[20]	training's multi_logloss: 0.341583	valid_1's multi_logloss: 0.446205
[30]	training's multi_logloss:

C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1

C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1

Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.504584	valid_1's multi_logloss: 0.518332
[20]	training's multi_logloss: 0.442463	valid_1's multi_logloss: 0.460001
[30]	training's multi_logloss: 0.397771	valid_1's multi_logloss: 0.421388
[40]	training's multi_logloss: 0.375836	valid_1's multi_logloss: 0.402632
[50]	training's multi_logloss: 0.35957	valid_1's multi_logloss: 0.393559
[60]	training's multi_logloss: 0.347835	valid_1's multi_logloss: 0.390178
[70]	training's multi_logloss: 0.339402	valid_1's multi_logloss: 0.389021
[80]	training's multi_logloss: 0.3325	valid_1's multi_logloss: 0.389238
Early stopping, best iteration is:
[72]	training's multi_logloss: 0.338262	valid_1's multi_logloss: 0.388452
Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.469811	valid_1's multi_logloss: 0.489096
[20]	training's multi_logloss: 0.394762	valid_1's multi_logloss: 0.429424
[30]	training's multi_logloss: 0

...

Early stopping, best iteration is:
[37]	training's multi_logloss: 0.287025	valid_1's multi_logloss: 0.400438
Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.458235	valid_1's multi_logloss: 0.49686
[20]	training's multi_logloss: 0.366607	valid_1's multi_logloss: 0.431544
[30]	training's multi_logloss: 0.315999	valid_1's multi_logloss: 0.4067
[40]	training's multi_logloss: 0.283334	valid_1's multi_logloss: 0.397133
[50]	training's multi_logloss: 0.259117	valid_1's multi_logloss: 0.393696
[60]	training's multi_logloss: 0.240206	valid_1's multi_logloss: 0.394252
Early stopping, best iteration is:
[53]	training's multi_logloss: 0.253349	valid_1's multi_logloss: 0.392395
Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.556518	valid_1's multi_logloss: 0.558184
[20]	training's multi_logloss: 0.490024	valid_1's multi_logloss: 0.494956
[30]	training's multi_logloss: 0.455995	valid_1's multi_logloss: 0.4641

[I 2023-06-19 21:52:28,498] Trial 33 finished with value: 0.3756202124969309 and parameters: {'max_bin': 419, 'num_leaves': 108, 'lambda_l1': 3.432187077228294e-06, 'lambda_l2': 0.0018953590604522356, 'feature_fraction': 0.4383731596725887, 'bagging_fraction': 0.6913765575607738, 'bagging_freq': 98, 'min_child_samples': 89}. Best is trial 33 with value: 0.3756202124969309.
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2':

[90]	training's multi_logloss: 0.33346	valid_1's multi_logloss: 0.383397
[100]	training's multi_logloss: 0.326404	valid_1's multi_logloss: 0.381218
[110]	training's multi_logloss: 0.319815	valid_1's multi_logloss: 0.379365
[120]	training's multi_logloss: 0.313829	valid_1's multi_logloss: 0.37766
[130]	training's multi_logloss: 0.308214	valid_1's multi_logloss: 0.376721
[140]	training's multi_logloss: 0.303737	valid_1's multi_logloss: 0.375693
[150]	training's multi_logloss: 0.299892	valid_1's multi_logloss: 0.376115
Early stopping, best iteration is:
[143]	training's multi_logloss: 0.302327	valid_1's multi_logloss: 0.37562
Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.463951	valid_1's multi_logloss: 0.482063
[20]	training's multi_logloss: 0.399772	valid_1's multi_logloss: 0.427731
[30]	training's multi_logloss: 0.369063	valid_1's multi_logloss: 0.405843
[40]	training's multi_logloss: 0.350475	valid_1's multi_logloss: 0.395573
[50]	trainin

C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
C:\Users\kabeo\AppData\Local\Temp\ipykernel_1572\3341545109.py:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1

Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.472295	valid_1's multi_logloss: 0.498502
[20]	training's multi_logloss: 0.406429	valid_1's multi_logloss: 0.447621
[30]	training's multi_logloss: 0.374297	valid_1's multi_logloss: 0.423043
[40]	training's multi_logloss: 0.356592	valid_1's multi_logloss: 0.410435
[50]	training's multi_logloss: 0.345717	valid_1's multi_logloss: 0.403504
[60]	training's multi_logloss: 0.338599	valid_1's multi_logloss: 0.399707
[70]	training's multi_logloss: 0.333365	valid_1's multi_logloss: 0.397423
[80]	training's multi_logloss: 0.329347	valid_1's multi_logloss: 0.396557
[90]	training's multi_logloss: 0.326592	valid_1's multi_logloss: 0.396556
Early stopping, best iteration is:
[82]	training's multi_logloss: 0.328705	valid_1's multi_logloss: 0.39652
Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.462285	valid_1's multi_logloss: 0.496402
[20]	training's multi_logloss:

In [66]:
study.best_params

{'max_bin': 419,
 'num_leaves': 108,
 'lambda_l1': 3.432187077228294e-06,
 'lambda_l2': 0.0018953590604522356,
 'feature_fraction': 0.4383731596725887,
 'bagging_fraction': 0.6913765575607738,
 'bagging_freq': 98,
 'min_child_samples': 89}

In [67]:
params = {
    'objective': 'multiclass',
    'verbose': -1,
    'num_class': 3,
    'max_bin' : study.best_params['max_bin'],
    'learning_rate' : 0.05,
    'num_leaves' : study.best_params['num_leaves'],
    'metric' : 'multi_logloss',
    'lambda_l1': study.best_params['lambda_l1'],
    'lambda_l2': study.best_params['lambda_l2'],
    'feature_fraction': study.best_params['feature_fraction'],
    'bagging_fraction': study.best_params['bagging_fraction'],
    'bagging_freq': study.best_params['bagging_freq'],
    'min_child_samples': study.best_params['min_child_samples']
}

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

model = lgb.train(params,lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  verbose_eval=10,
                  num_boost_round=1000,
                  early_stopping_rounds=10)
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 0.53889	valid_1's multi_logloss: 0.542051
[20]	training's multi_logloss: 0.464411	valid_1's multi_logloss: 0.473401
[30]	training's multi_logloss: 0.424115	valid_1's multi_logloss: 0.439256
[40]	training's multi_logloss: 0.396632	valid_1's multi_logloss: 0.416869
[50]	training's multi_logloss: 0.378813	valid_1's multi_logloss: 0.4062
[60]	training's multi_logloss: 0.362579	valid_1's multi_logloss: 0.39522
[70]	training's multi_logloss: 0.351304	valid_1's multi_logloss: 0.390167
[80]	training's multi_logloss: 0.342395	valid_1's multi_logloss: 0.387488
[90]	training's multi_logloss: 0.33346	valid_1's multi_logloss: 0.383397
[100]	training's multi_logloss: 0.326404	valid_1's multi_logloss: 0.381218
[110]	training's multi_logloss: 0.319815	valid_1's multi_logloss: 0.379365
[120]	training's multi_logloss: 0.313829	valid_1's multi_logloss: 0.37766
[130]	training's multi_logloss: 0.308214	valid_1's mul

C:\Users\kabeo\AppData\Roaming\Python\Python310\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\kabeo\AppData\Roaming\Python\Python310\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\kabeo\AppData\Roaming\Python\Python310\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\kabeo\AppData\Roaming\Python\Python310\site-pack

In [68]:
y_pred.shape

(400, 3)

In [69]:
y_pred_max = np.argmax(y_pred, axis=1)

In [70]:
sample_submit = pd.read_csv("sample_submit.csv",header=None, names=['id','charges'])

In [71]:
sample_submit['charges'] = y_pred_max

In [72]:
pd.DataFrame(sample_submit).to_csv('hold-outOptuna3_.csv', index=False, header=False)